# Imports

In [20]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
from pandas_profiling import ProfileReport

In [22]:
conn = sqlite3.connect("data\data_wildfire_usa.sqlite")

In [23]:
#fires = pd.read_sql_query("SELECT * FROM fires". conn)
# fires est un tableau comprenant des données sur les incendies de forêt pour la période 1992-2015 compilées à partir des systèmes de déclaration fédéraux, étatiques et locaux des États-Unis.

In [24]:
#fires.info()

In [25]:
#fires.head()

# Description des variables + notes à chaud

1.   OBJECTID : ID : donnée a priori non nécessaire pour notre étude

2.   FOD_ID : Identificateur unique global qui ressemble à l'OBJECTID 

3.   FPA_ID : Identificateur unique nécessaires pour remonter jusqu'à l'enregistrement d'origine 

4.   SOURCE_SYSTEM_TYPE : Type de données source ou système d'où provient l'enregistrement (fédéral, non fédéral ou interagence) 

5.   SOURCE_SYSTEM : Nom ou autre identifiant de la données source ou système d'où provient l'enregistrement 

6.   NWCG_REPORTING_AGENCY : Active National Wildlife Coordinating Group = Groupe national de coordination de la faune. Identifiant d'unité pour l'agence qui prépare le rapport d'incendie.
  *   BIA = Bureau of Indian Affairs 
  *   BLM = Bureau of Land Management
  *   BOR = Bureau of Reclamation
  *   DOD = Department of Defense
  *   DOE = Department of Energy
  *   FS = Forest Service
  *   FWS = Fish and Wildlife Service
  *   IA = Interagency Organization
  *   NPS = National Park Service
  *   ST/C&L = State, County, or Local Organization
  *   TRIBE = Tribal Organization

7.   NWCG_REPORTING_UNIT_ID : Identifiant d'unité NWCG actif pour l'unité préparant le rapport d'incendie 

8.   NWCG_REPORTING_UNIT_NAME : Nom d'unité NWCG actif pour l'unité préparant le rapport d'incendie

9.   SOURCE_REPORTING_UNIT : Code de l'unité d'agence préparant le rapport d'incendie, basé sur le code/nom dans les données source 

10.  SOURCE_REPORTING_UNIT_NAME : Nom de l'unité de l'agence déclarante préparant le rapport d'incendie, basé sur le code/nom des données source 

11.  LOCAL_FIRE_REPORT_ID : Numéro ou code qui identifie de manière unique un rapport d'incident pour une unité de déclaration particulière et une année civile particulière 

12.  LOCAL_INCIDENT_ID : Numéro ou code qui identifie de manière unique un incident pour une organisation locale de gestion des incendies particulière au cours d'une année civile donnée 

13.  FIRE_CODE : code utilisé au sein de la communauté inter-agences des feux de forêt pour suivre et compiler les informations sur les coûts de la suppression des incendies d'urgence

14.  FIRE_NAME : Nom de l'incident, tiré du rapport d'incendie (primaire) ou du rapport ICS-209 (secondaire)

15.  ICS_209_INCIDENT_NUMBER : Identifiant d'incident (événement), issu du rapport ICS-209. 

16.  ICS_209_NAME : Nom de l'incident, tiré du rapport ICS-209. 

17.  MTBS_ID : Identifiant d'incident, à partir de l'ensemble de données de périmètre MTBS. 

18.  MTBS_FIRE_NAME : Nom de l'incident, issu de l'ensemble de données de périmètre MTBS. 

19.  COMPLEX_NAME : Nom du complexe sous lequel l'incendie a finalement été maîtrisé, lorsqu'il est discernable. 

20.  FIRE_YEAR : Année civile au cours de laquelle l'incendie a été découvert ou confirmé. 

21.  DISCOVERY_DATE : Date à laquelle l'incendie a été découvert ou confirmé. 

22.  DISCOVERY_DOY : Jour de l'année où l'incendie a été découvert ou confirmé. 

23.  DISCOVERY_TIME : Heure de la journée à laquelle l'incendie a été découvert ou confirmé. 
            
24.  STAT_CAUSE_CODE : Code de la cause (statistique) de l'incendie. **=> Donnée non nécessaire si on a le nom. Nécessaire seulement si cette donnée nous permet de combler des valeurs manquantes dans la colonne STAT_CAUSE_DESCR**

25.  STAT_CAUSE_DESCR : Description de la cause (statistique) de l'incendie. 

26.  CONT_DATE : Date à laquelle l'incendie a été déclaré maîtrisé ou autrement maîtrisé (mm/jj/aaaa où mm=mois, jj=jour et aaaa=année). 

27.  CONT_DOY : Jour de l'année où l'incendie a été déclaré maîtrisé ou autrement maîtrisé. 

28.  CONT_TIME : Heure de la journée à laquelle l'incendie a été déclaré maîtrisé ou maîtrisé (hhmm où hh=heure, mm=minutes). *

29.  FIRE_SIZE : Estimation des acres dans le périmètre final du feu. (1 acre anglo-saxon = 40,47 ares). **A FAIRE : voir s'il y a corrélation entre la taille du feu et le temps**

30.  FIRE_SIZE_CLASS : Code de la taille du feu basé sur le nombre d'acres dans les dépenses finales du périmètre d'incendie
  *   A = > 0 et <= 0,25 acre
  *   B = 0,26 à 9,9 acres
  *   C = 10,0 à 99,9 acres
  *   D = 100 à 299 acres
  *   E = 300 à 999 acres
  *   F = 1000 à 4999 acres
  *   G = 5000+ acres

31.  LATITUDE : Latitude (NAD83) pour l'emplacement ponctuel de l'incendie (degrés décimaux). 

32.  LONGITUDE : Longitude (NAD83) pour l'emplacement ponctuel de l'incendie (degrés décimaux). 

33.  OWNER_CODE : Code du propriétaire principal ou de l'entité responsable de la gestion du terrain au point d'origine de l'incendie au moment de l'incident. **=> Donnée non nécessaire si on a le nom. Nécessaire seulement si cette donnée nous permet de combler des valeurs manquantes dans la colonne OWNER_DESCR**

34.  OWNER_DESCR : Nom du propriétaire principal ou de l'entité responsable de la gestion du terrain au point d'origine de l'incendie au moment de l'incident

35.  STATE : Code alphabétique à deux lettres de l'état dans lequel l'incendie a brûlé (ou s'est déclaré), basé sur la désignation nominale dans le rapport d'incendie. **Correspondance à faire avec la liste ci-dessous**

(AK=Alaska, AL=Alabama, AR=Arkansas, AZ=Arizona, CA=California, CO=Colorado, CT=Connecticut, DE=Delaware, DHS=Department of Homeland Security, DOC= Department of Commerce, DOD=Department of Defense, DOE=Department of Energy, DOI= Department of Interior, DOL=Department of Labor, FL=Florida, GA=Georgia, IA=Iowa, IA/GC=Non-Departmental Agencies, ID=Idaho, IL=Illinois, IN=Indiana, KS=Kansas, KY=Kentucky, LA=Louisiana, MA=Massachusetts, MD=Maryland, ME=Maine, MI=Michigan, MN=Minnesota, MO=Missouri, MS=Mississippi, MT=Montana, NC=North Carolina, NE=Nebraska, NG=Non-Government, NH=New Hampshire, NJ=New Jersey, NM=New Mexico, NV=Nevada, NY=New York, OH=Ohio, OK=Oklahoma, OR=Oregon, PA=Pennsylvania, PR=Puerto Rico, RI=Rhode Island, SC=South Carolina, SD=South Dakota, ST/L=State or Local Government, TN=Tennessee, Tribe=Tribe, TX=Texas, USDA=Department of Agriculture, UT=Utah, VA=Virginia, VI=U. S. Virgin Islands, VT=Vermont, WA=Washington, WI=Wisconsin, WV=West Virginia, WY=Wyoming).

36.  COUNTY : Comté, ou équivalent, dans lequel l'incendie s'est déclaré (ou a pris naissance), en fonction de la désignation nominale dans le rapport d'incendie.

37.  FIPS_CODE : Code à trois chiffres de la publication FIPS (Federal Information Process Standards) 6-4 pour la représentation des comtés et des entités équivalentes. **=> Donnée non nécessaire si on a le nom. Nécessaire seulement si cette donnée nous permet de combler des valeurs manquantes dans la colonne FIPS_NAME**

38.  FIPS_NAME : Nom du comté de la publication FIPS 6-4 pour la représentation des comtés et des entités équivalentes

39.  Shape : donnée inutile

# Statistiques

In [26]:
# Statistiques au format html
#prof = ProfileReport(fires)
#prof.to_file(output_file='rapport.html')

In [27]:
# Affichage des statistiques dans le notebook via la librairie pandas_profiling
#profile = ProfileReport(fires, title='Analyse du fichier WildFire', html={'style':{'full_width':True}})
#profile.to_notebook_iframe()

Suite à la méthode ProfileReport, nous pouvons déclarer :
- Sur 39 entrées : 14 sont des variables de tyme numérique, 24 de type catégorielle et 1 ne peut être exploitée
- Il y a 27% de valeurs manquantes dans le DataFrame
- Etant donné la part de valeurs manquantes dans les colonnes ICS_209_INCIDENT_NUMBER (98.6%), ICS_209_NAME (98.6%), MTBS_ID (99.4%), MTBS_FIRE_NAME (99.4%), COMPLEX_NAME (99.7%), ces colonnes ne seront pas utilisées pour l'étude.
- Il n'y a pas de doubons parfaits : les doublons seront à analyser en détail par colonne
- Certains colonnes sont à haute cardinalité, c'est à dire qu'elles ont des contraintes (uniques) qui nous empêchent de les regrouper. 
- Les colonnes OBJECTID, FOD_ID et FPA_ID ont des valeurs uniques. FPA_ID a des valeurs disctintes à 99.9%, il faut analyser ce point.
- Pour certaines données, nous avons à la fois des codes et des libellés. Il faudra analyser s'il y a un lien entre ces données et si nous pouvons remplacer des valeurs manquantes.
- Certains formats de date ne sont pas lisibles

# Data Cleaning

In [28]:
# Le chargement du fichier initial est nécessaire dans cette cellule pour éviter tout message d'erreur si la cellule est ré-exécutée
fires = pd.read_sql_query("SELECT * FROM fires", conn)
# Suppression de la colonne Shape
fires = fires.drop(['Shape'], axis = 1)
# Renommage des colonnes : DAY à la place de DOY
fires = fires.rename(columns = {'DISCOVERY_DOY' : 'DISCOVERY_DAY', 'CONT_DOY' : 'CONT_DAY'})
fires.head()


,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,0.10,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,0.25,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,0.10,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,0.10,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,0.10,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine


## Etude des variables identiques ou similaires

In [29]:
# Est ce que la variable OBJECTID = FOD_ID ?
(fires.OBJECTID == fires.FOD_ID).value_counts()
# Les 2 colonnes semblent être identiques mais l'ID de FOD_ID est décalé à partir de l'index 72. 
# A priori ces données ne sont pas nécessaires pour l'étude. On ne s'en préoccupera pas.

False    1880393
True          72
dtype: int64

In [30]:
# Est ce qu'une valeur donnée dans la colonne SOURCE_SYSTEM_TYPE correspond à une valeur spécifique de la colonne SOURCE_SYSTEM ?
source_system = (fires['SOURCE_SYSTEM_TYPE']) + '_' + (fires['SOURCE_SYSTEM'])
print(source_system.duplicated().value_counts(), '\n')
print('Résultat comparaison des 2 colonnes :\n', source_system.drop_duplicates(keep = 'first').values)
# 39 observations ne sont pas identiques. On peut se demander s'il s'agit d'erreurs.

True     1880426
False         39
dtype: int64 

Résultat comparaison des 2 colonnes :
 ['FED_FS-FIRESTAT' 'FED_DOI-WFMI' 'FED_FWS-FMIS' 'NONFED_FA-NFIRS'
 'NONFED_ST-NASF' 'NONFED_ST-AZAZS' 'NONFED_ST-MOMOS' 'INTERAGCY_IA-AKACC'
 'NONFED_ST-MTMTS' 'NONFED_ST-SCSCS' 'NONFED_ST-COCOS' 'NONFED_ST-MEMES'
 'NONFED_ST-ORORS' 'NONFED_ST-MSMSS' 'NONFED_ST-NMNMS' 'NONFED_ST-SDSDS'
 'NONFED_ST-UTUTS' 'NONFED_ST-WAWAS' 'NONFED_ST-WYWYS' 'NONFED_ST-OKOKS'
 'NONFED_ST-WIWIS' 'NONFED_ST-MIMIS' 'NONFED_ST-KYKYS' 'NONFED_ST-ALALS'
 'NONFED_ST-ARARS' 'NONFED_ST-GAGAS' 'NONFED_ST-LALAS' 'NONFED_ST-TNTNS'
 'NONFED_ST-VAVAS' 'NONFED_ST-FLFLS' 'NONFED_ST-NCNCS'
 'INTERAGCY_IA-ICS209' 'NONFED_ST-CACDF' 'NONFED_ST-CTCTS'
 'NONFED_ST-NENES' 'NONFED_IA-ICS209' 'NONFED_ST-TXTXS'
 'INTERAGCY_IA-PRIITF' 'INTERAGCY_IA-HIWMO']


In [31]:
# Est ce qu'une valeur donnée dans la colonne NWCG_REPORTING_UNIT_ID correspond à une valeur spécifique de la colonne NWCG_REPORTING_UNIT_NAME ?
nwcg_reporting = pd.DataFrame(fires['NWCG_REPORTING_UNIT_ID']).join(fires['NWCG_REPORTING_UNIT_NAME'])
nwcg_reporting.duplicated().value_counts()
# 1640 observations ne sont pas identiques. On peut se demander s'il s'agit d'erreur.

True     1878825
False       1640
dtype: int64

In [32]:
# Est ce qu'une valeur donnée dans la colonne FIRE_CODE correspond à une valeur spécifique de la colonne FIRE_NAME ?
fires_code_name = pd.DataFrame(fires['FIRE_CODE']).join(fires['FIRE_NAME'])
print(fires_code_name.duplicated().value_counts(), '\n')
group_by = fires_code_name.groupby(['FIRE_NAME'], as_index = False).value_counts()
print('Résultat comparaison des 2 colonnes :')
group_by[group_by['count'] > 1].sort_values(by = 'count', ascending = False)
# On voit bien qu'un code ne correspond pas systématiquement à un seul nom de feu. Un même code peut posséder plusieurs noms et inversement. 


True     1187560
False     692905
dtype: int64 

Résultat comparaison des 2 colonnes :


,FIRE_NAME,FIRE_CODE,count
160374,LOCAL,D44Z,496
264952,STATE,D44Z,263
160558,LOCAL FIRE,D44Z,159
265286,STATE FIRE,D44Z,84
160619,LOCAL VEGETATION,D44Z,28
...,...,...,...
120741,HACKOMILLER,D44Z,2
120362,GURULE,8000,2
120361,GURULE,EKV9,2
120297,GUNSIGHT,EKW0,2


In [33]:
# Est ce qu'une valeur donnée dans la colonne SOURCE_REPORTING_UNIT correspond à une valeur spécifique de la colonne SOURCE_REPORTING_UNIT_NAME ?
source_reporting = pd.DataFrame(fires['SOURCE_REPORTING_UNIT']).join(fires['SOURCE_REPORTING_UNIT_NAME'])
print(source_reporting.duplicated().value_counts(), '\n')
group_by_source = source_reporting.groupby(['SOURCE_REPORTING_UNIT'], as_index = False).value_counts()
print('Résultat comparaison des 2 colonnes :')
group_by_source[group_by_source['count'] > 1].sort_values(by = 'SOURCE_REPORTING_UNIT')
# Pour cette variable également on s'aperçoit qu'un code ne correspond pas à un seul nom et inversement.

True     1873584
False       6881
dtype: int64 

Résultat comparaison des 2 colonnes :


,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,count
1,00203,FILLMORE FIRE DEPT,5
2,00206,SAVANNAH RFD,11
3,00307,FAIRFAX RFPD,15
4,00308,ROCK PORT FIRE DEPT,5
5,00501,CASSVILLE FORESTRY,23
...,...,...,...
6876,WYWRA,Wind River Agency,1185
6877,WYWSFD,Wyoming State Forestry,1947
6878,WYWYS,Wyoming State Forestry,2143
6879,WYYNP,Yellowstone National Park,515


In [34]:
# Est ce qu'une valeur donnée dans la colonne STAT_CAUSE_CODE correspond à une valeur spécifique de la colonne STAT_CAUSE_DESCR ?
stat_cause = pd.DataFrame(fires['STAT_CAUSE_CODE']).join(fires['STAT_CAUSE_DESCR'])
print(stat_cause.duplicated().value_counts(), '\n')
group_by_stat = stat_cause.groupby(['STAT_CAUSE_CODE'], as_index = False).value_counts()
print('Résultat comparaison des 2 colonnes :')
group_by_stat
# La variable STAT_CAUSE_CODE correspond à une donnée de STAT_CAUSE_DESCR

True     1880452
False         13
dtype: int64 

Résultat comparaison des 2 colonnes :


,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,count
0,1.0,Lightning,278468
1,2.0,Equipment Use,147612
2,3.0,Smoking,52869
3,4.0,Campfire,76139
4,5.0,Debris Burning,429028
5,6.0,Railroad,33455
6,7.0,Arson,281455
7,8.0,Children,61167
8,9.0,Miscellaneous,323805
9,10.0,Fireworks,11500


In [74]:
# Est ce qu'une valeur donnée dans la colonne OWNER_CODE correspond à une valeur spécifique de la colonne OWNER_DESCR ?
owner = pd.DataFrame(fires['OWNER_CODE']).join(fires['OWNER_DESCR'])
print(owner.duplicated().value_counts(), '\n')
group_by_owner = owner.groupby(['OWNER_CODE'], as_index = False).value_counts()
print('Résultat comparaison des 2 colonnes :')
group_by_owner
# La variable OWNER_CODE correspond à une donnée de OWNER_DESCR

True     1880449
False         16
dtype: int64 

Résultat comparaison des 2 colonnes :


,OWNER_CODE,OWNER_DESCR,count
0,0.0,FOREIGN,15
1,1.0,BLM,63278
2,2.0,BIA,106819
3,3.0,NPS,17524
4,4.0,FWS,12191
5,5.0,USFS,188338
6,6.0,OTHER FEDERAL,6452
7,7.0,STATE,30790
8,8.0,PRIVATE,314822
9,9.0,TRIBAL,8952


In [72]:
# Est ce que la variable FIRE_SIZE_CLASS correspond à la variable FIRE_SIZE ?
# Création d'une colonne
fires['FIRE_SIZE_CLASS_TEST'] = 0
# Création d'une liste de condition
conditionlist = [
    (fires['FIRE_SIZE'] <= 0.25),
    (fires['FIRE_SIZE'] >0.25) & (fires['FIRE_SIZE']<10),
    (fires['FIRE_SIZE'] >=10) & (fires['FIRE_SIZE'] <100),
    (fires['FIRE_SIZE'] >=100) & (fires['FIRE_SIZE'] <300),
    (fires['FIRE_SIZE'] >=300) & (fires['FIRE_SIZE'] <1000),
    (fires['FIRE_SIZE'] >=1000) & (fires['FIRE_SIZE'] <5000),
    (fires['FIRE_SIZE']>=5000)]

# Création d'une liste de choix
choicelist = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

# Comparaison avec la colonne existante
fires['FIRE_SIZE_CLASS_TEST'] = np.select(conditionlist, choicelist)
(fires['FIRE_SIZE_CLASS_TEST'] == fires['FIRE_SIZE_CLASS']).value_counts()


True    1880465
dtype: int64

La colonne FIRE_SIZE_CLASS est conforme à la colonne FIRE_SIZE

## Examen des doublons

L'examen des doublons sur l'ensemble des colonnes du DataFrame peut se faire directement avec les résulats de ProfileReport. Cependant, pour plus de clareté nous allons détailler certains points

In [39]:
# Création d'un dictionnaire regroupant les colonnes et le nombre de doublons par colonne
doublons = {}
for col in fires.columns :
    duplicates = fires[col].duplicated().sum()
    doublons[col] = duplicates

doublons

{'OBJECTID': 0,
 'FOD_ID': 0,
 'FPA_ID': 3,
 'SOURCE_SYSTEM_TYPE': 1880462,
 'SOURCE_SYSTEM': 1880427,
 'NWCG_REPORTING_AGENCY': 1880454,
 'NWCG_REPORTING_UNIT_ID': 1878825,
 'NWCG_REPORTING_UNIT_NAME': 1878830,
 'SOURCE_REPORTING_UNIT': 1875473,
 'SOURCE_REPORTING_UNIT_NAME': 1876024,
 'LOCAL_FIRE_REPORT_ID': 1866956,
 'LOCAL_INCIDENT_ID': 1314550,
 'FIRE_CODE': 1708018,
 'FIRE_NAME': 1386831,
 'ICS_209_INCIDENT_NUMBER': 1857727,
 'ICS_209_NAME': 1860891,
 'MTBS_ID': 1869983,
 'MTBS_FIRE_NAME': 1872331,
 'COMPLEX_NAME': 1879048,
 'FIRE_YEAR': 1880441,
 'DISCOVERY_DATE': 1871699,
 'DISCOVERY_DAY': 1880099,
 'DISCOVERY_TIME': 1879024,
 'STAT_CAUSE_CODE': 1880452,
 'STAT_CAUSE_DESCR': 1880452,
 'CONT_DATE': 1871704,
 'CONT_DAY': 1880098,
 'CONT_TIME': 1879023,
 'FIRE_SIZE': 1866828,
 'FIRE_SIZE_CLASS': 1880458,
 'LATITUDE': 986404,
 'LONGITUDE': 882929,
 'OWNER_CODE': 1880449,
 'OWNER_DESCR': 1880449,
 'STATE': 1880413,
 'COUNTY': 1877009,
 'FIPS_CODE': 1880179,
 'FIPS_NAME': 1878766,
 '

Le résultat de ce dictionnaire nous indique plusieurs choses :
1. Pour la plupart des variables, il n'existe que très peu de données distinctes car il y a beaucoup de doublons.
2. Pour les variables uniques :
    - telles que OBJECTID et FOD_ID, il n'existe pas de doublons. Il n'y a donc pas d'erreur dans ces données car elles sont uniques (en lien avec les résultats de la méthode ProfileReport).
    - En revanche, pour la variable FPA_ID, nous remarquons qu'il y a 3 doublons. Ces erreurs doivent être corrigées car FPA_ID est une donnée censée être unique 

In [40]:
# Identification et correction des doublons de FPA_ID
fpa_doublon = fires[fires['FPA_ID'].duplicated(keep = False) == True]
fpa_doublon
# Les 3 doublons concernent les FPA_ID suivants : FS-1452833 ; ICS209_2009_KS-DDQ-128 et SFO-2015CACDFLNU003791


,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,FIRE_SIZE_CLASS_TEST
21986,21987,22093,FS-1452833,FED,FS-FIRESTAT,FS,USNMCIF,Cibola National Forest,0303,Cibola National Forest,...,B,35.312778,-107.593056,5.0,USFS,NM,31,031,McKinley,0
1065673,1065674,1300088,ICS209_2009_KS-DDQ-128,INTERAGCY,IA-ICS209,DOD,USKSFRQ,Fort Riley Kansas,KSDDQ,Fort Riley,...,F,39.234444,-96.830278,6.0,OTHER FEDERAL,KS,Riley,161,Riley,0
1565829,1565830,201432072,FS-1452833,FED,FS-FIRESTAT,FS,USNMCIF,Cibola National Forest,0303,Cibola National Forest,...,A,35.337222,-107.779444,5.0,USFS,NM,031,031,McKinley,0
1634979,1634980,201750002,ICS209_2009_KS-DDQ-128,INTERAGCY,IA-ICS209,DOD,USKSFRQ,Fort Riley Kansas,KSDDQ,Fort Riley,...,F,39.220000,-96.940000,6.0,OTHER FEDERAL,KS,Riley,161,Riley,0
1825692,1825693,300245030,SFO-2015CACDFLNU003791,NONFED,ST-NASF,ST/C&L,USCALNU,Sonoma-Lake Napa Unit,CALNU,Sonoma-Lake Napa Unit,...,B,38.715883,-122.994933,15.0,UNDEFINED FEDERAL,CA,SONOMA,097,Sonoma,0
1870332,1870333,300306586,SFO-2015CACDFLNU003791,NONFED,ST-NASF,ST/C&L,USCALNU,Sonoma-Lake Napa Unit,CALNU,Sonoma-Lake Napa Unit,...,A,38.342004,-121.958596,14.0,MISSING/NOT SPECIFIED,CA,SOLANO,095,Solano,0


In [41]:
# Examen des doublons
fpa_doublon.sort_values(by = 'FPA_ID', inplace = True)
for col in fpa_doublon.columns :
    if any(fpa_doublon.groupby(['FPA_ID'])[col].nunique()>1)  :
        print(fpa_doublon[['FPA_ID', col]])
        print('='*50)
# Au vu du nombre de donnée en doublon, nous pouvons supprimer les données qu'on ne peut pas expliquer ou bien les laisser.
# 3 doublons sur 1800000 observation ont peu d'impact.

                         FPA_ID  OBJECTID
21986                FS-1452833     21987
1565829              FS-1452833   1565830
1065673  ICS209_2009_KS-DDQ-128   1065674
1634979  ICS209_2009_KS-DDQ-128   1634980
1825692  SFO-2015CACDFLNU003791   1825693
1870332  SFO-2015CACDFLNU003791   1870333
                         FPA_ID     FOD_ID
21986                FS-1452833      22093
1565829              FS-1452833  201432072
1065673  ICS209_2009_KS-DDQ-128    1300088
1634979  ICS209_2009_KS-DDQ-128  201750002
1825692  SFO-2015CACDFLNU003791  300245030
1870332  SFO-2015CACDFLNU003791  300306586
                         FPA_ID LOCAL_FIRE_REPORT_ID
21986                FS-1452833                   46
1565829              FS-1452833                  040
1065673  ICS209_2009_KS-DDQ-128                 None
1634979  ICS209_2009_KS-DDQ-128                 None
1825692  SFO-2015CACDFLNU003791                 None
1870332  SFO-2015CACDFLNU003791                 None
                         FPA_ID LO

C:\Users\Enedis\AppData\Local\Temp\ipykernel_8540\1141412449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fpa_doublon.sort_values(by = 'FPA_ID', inplace = True)


## Examen du type/format de données

In [42]:
# Vérification de la colonne FIRE_YEAR
fires['FIRE_YEAR'].value_counts().sort_index()

1992     67975
1993     61989
1994     75955
1995     71472
1996     75574
1997     61450
1998     68370
1999     89363
2000     96416
2001     86587
2002     75656
2003     68261
2004     69279
2005     88604
2006    114004
2007     95573
2008     85378
2009     78325
2010     79889
2011     90552
2012     72769
2013     64780
2014     67753
2015     74491
Name: FIRE_YEAR, dtype: int64

In [43]:
# La colonne DISCOVERY_DATE n'est pas au bon format
# Changement du format de date de DISCOVERY_DATE
fires['DATE'] = pd.to_datetime(fires['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
# Vérification du format
print(pd.DataFrame(fires['DATE']).join(fires['FIRE_YEAR']))

# La colonne CONT_DATE n'est pas au bon format
# Changement du format de date de CONT_DATE
fires['CONT_DATE_COR'] = pd.to_datetime(fires['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
# Vérification du format
fires['CONT_DATE_COR']

              DATE  FIRE_YEAR
0       2005-02-02       2005
1       2004-05-12       2004
2       2004-05-31       2004
3       2004-06-28       2004
4       2004-06-28       2004
...            ...        ...
1880460 2015-09-26       2015
1880461 2015-10-05       2015
1880462 2015-05-02       2015
1880463 2015-10-14       2015
1880464 2015-03-14       2015

[1880465 rows x 2 columns]


0         2005-02-02
1         2004-05-12
2         2004-05-31
3         2004-07-03
4         2004-07-03
             ...    
1880460   2015-09-26
1880461          NaT
1880462          NaT
1880463          NaT
1880464          NaT
Name: CONT_DATE_COR, Length: 1880465, dtype: datetime64[ns]

# Examen des causes des incendies

In [44]:
#plt.figure(figsize=(10, 8))
#plt.barh(fires['STAT_CAUSE_DESCR'].value_counts(), color = 'darkslateblue', width = 0.6)
#plt.title(label = 'XXXX', size = 'xx-large', weight = 'bold', color = 'crimson')
#plt.xlabel('XXXX', weight = 'bold')
#plt.show()

Il y a 13 causes répertoriées dans l'ensemble de données. 
Les incendies sont-ils plus probables certains jours de la semaine ?

In [45]:
#fires['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='navy')
#plt.show()

Les incendies sont légèrement plus susceptibles de se déclencher le week-end. 
On peut supposer que les incendies sont déclenchés par des personnes négligentes : feux de camp, cigarettes, incendies malveillants.
Il est peu probable que les incendies augmentant le week-end proviennent de causes naturelles, comme la foudre par exemple. Nous pouvons tester cela :

In [46]:
#fires_lightning = fires[fires['STAT_CAUSE_DESCR']=='Lightning']
#fires_lightning['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='navy')
#plt.show()

Il y a quelques petites variations, mais nous voyons que les feux se déclenchant le week-end sont moins élevés. 
Nous pouvons examiner si l'augmentation générale du week-end est due à des incendies malveillants
Incendie criminel :

In [47]:
#fires_arson = fires[fires['STAT_CAUSE_DESCR']=='Arson']
#fires_arson['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='navy')
#plt.show()

Cela montre une tendance plus significative.
Les incendies criminels sont plus probables le week-end que pendant la semaine, cela représente une augmentation d'environ 30 % de la moyenne des jours de la semaine. 

Quels États ont le plus d'incendies ? (pour éviter un trop grand nombre de données, nous pouvons nous concentrer sur les dix premiers)

In [48]:
#fires['STATE'].value_counts().head(n=10).plot(kind='barh',color='navy')
#plt.show()

CA et TX sont dans le top 3, ce sont de grands états avec des climats secs. En revanche, GA est également dans le top 3 on peut se demander pourquoi
Nous pouvons enquêter sur les causes des incendies dans ces 3 états et rechercher des différences. On extrait les données pour ces 3 états :

In [49]:
#df_CA = df[df['STATE']=='CA']
#df_GA = df[df['STATE']=='GA']
#df_TX = df[df['STATE']=='TX']

In [50]:
#fires_CA['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='navy',title='causes of fires for CA')
#plt.show()

In [51]:
#fires_GA['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='navy',title='causes of fires for GA')
#plt.show()

In [52]:
#fires_TX['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='navy',title='causes of fires for TX')
#plt.show()

Il existe des différences entre les états. 
Il existe deux manières de déterminer l'emplacement d'un incendie dans ce jeu de données - ETAT ou LATITUDE + LONGITUDE
Nous pouvons avoir une idée de la couverture géographique des données en traçant chaque paire de latitude et de longitude

In [53]:
#fires.plot(kind='scatter',x='LONGITUDE',y='LATITUDE',color='coral',alpha=0.3)
#plt.show()